In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/jobad"

 ad_detail_v1.json			      'qry_rel_machine learning.json'
 d2v_epoch100_vecsize15_field-abstract.model   qry_rel_v1a.json
'Job ads Design Document.gdoc'


# Set Up Learning Model

In [ ]:
# Get job ad data

import pandas as pd
df = pd.read_json('/content/drive/MyDrive/jobad/ad_detail_v1.json', lines=True)
df.head(5)

,title,abstract,content,metadata,ad_id
0,Senior Developer | SA,Readifarians are known for discovering the lat...,<p>Readify helps organizations innovate with t...,"{'location': 'Adelaide', 'workType': 'Full Tim...",38946054
1,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,This is a key role within a market leading Tra...,{'additionalSalaryText': '$110k - $120k p.a. +...,38856271
2,Level 2 Support Engineer,Seeking an experienced Level 2 Windows Support...,<p><strong>Summary</strong><br>ITtelligent is ...,"{'additionalSalaryText': '$55,000 - $69,999', ...",38901017
3,Senior JavaScript / Full-Stack Developer,"Our client, HazardCo, is seeking a JavaScript ...","<p>Kia Ora, New Zealand needs you!</p><p>We ar...","{'additionalSalaryText': '$90,000 - $130,000',...",38846084
4,CONTRACT Senior Front End Web Developer (Angul...,6 month rolling CONTRACT? Senior Web Developer...,<p>Our innovative client is looking for a CONT...,{'additionalSalaryText': '$80-$100 per hour 6 ...,38977270


In [ ]:
# Function: remove html tags from the description
import re

def remove_html_tags(text):    
    patn = re.compile('<.*?>')
    return re.sub(patn, '', text)

In [ ]:
# Extract specified field as model input

field='content'
#abstract_list = df["abstract"].tolist()
field_list = list(map(remove_html_tags, df[field]))
field_list[0]

"Readify helps organizations innovate with technology by providing high value software and digital services. Since 2001, Readify has a strong record of helping customers in a range of industries to develop and support innovative software projects. We use agile principles and practices to form autonomous, collaborative and cross-functional teams with our customers. Readify embraces open source software, both as consumers and producers, and offer a generous paid professional development scheme.The RoleAs a Senior Software Developer you'll provide our clients with an extremely high level of technical expertise and represent Readify as a technical expert in the wider community. You'll have the chance to make an impact in the development of complex, disruptive solutions for a wide range of organizations.You will be able to apply your passion for technology and problem solving, using the latest, cutting edge technologies whilst working alongside and learning from talented, knowledgeable and 

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Set up doc2vec model

max_epochs = 100
vec_size = 15
alpha = 0.025

tagged_data = [TaggedDocument(words=word_tokenize(_field.lower()), tags=[i]) for i, _field in enumerate(field_list)]
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

/usr/local/lib/python3.8/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
# Train the model

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0


<ipython-input-9-2e5e6a58b1f8>:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  epochs=model.iter)


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [ ]:
# Save the model

f_name = "/content/drive/MyDrive/jobad/d2v_epoch{}_vecsize{}_field-{}.model".format(max_epochs, vec_size, field)
model.save(f_name)
print("Model Saved in {}".format(f_name))

Model Saved in /content/drive/MyDrive/jobad/d2v_epoch100_vecsize15_field-content.model


In [ ]:
# Load the saved model

# from gensim.models.doc2vec import Doc2Vec

f_name = "/content/drive/MyDrive/jobad/d2v_epoch{}_vecsize{}_field-{}.model".format(max_epochs, vec_size, field)
model= Doc2Vec.load(f_name)



# Prepare Query Results

In [ ]:
# Input the query keywords

query = input("Input your query keywords:")

Input your query keywords:IT support


In [ ]:
# Infer the vector of the query keywords

test_data = word_tokenize(query.lower())
query_vec = model.infer_vector(test_data)
print("query_infer", query_vec)

query_infer [-0.01841908 -0.03865655  0.00033624  0.04543457  0.0087572  -0.03442594
 -0.05522738 -0.02805052  0.02732532 -0.02994857  0.00319669 -0.03071992
 -0.05764436  0.01064734 -0.02035608]


In [ ]:
# Find most similar doc using tags

# similar_doc = model.docvecs.most_similar(1)
# print(similar_doc)


### find vector of doc in training data using tags
# print(model.docvecs[1])

[(2756, 0.8457973599433899), (4570, 0.8287819623947144), (2719, 0.8177149891853333), (4381, 0.8114129900932312), (4717, 0.8094561696052551), (3341, 0.7994657158851624), (3590, 0.7958623170852661), (4586, 0.7777628302574158), (1038, 0.7727929949760437), (1893, 0.7715567350387573)]
[ 5.9261527   0.70213836 -8.15227     4.871992   -7.7033625   2.834238
  1.3195573   2.6917422  -1.9054825   0.5035495  -0.37564647 -5.9801817
  0.16827598 -3.5557318   5.0641294 ]


In [ ]:
# Show model vocabulary
# model.wv.vocab

In [ ]:
# Calculate the most similarities of job ads relevant to query keywords

num_top_search = len(model.docvecs)

sims=model.docvecs.most_similar([query_vec], topn=num_top_search)
sims[:10]

[(4058, 0.8126524090766907),
 (1626, 0.7296479344367981),
 (3635, 0.7192395329475403),
 (3540, 0.7191036343574524),
 (4891, 0.7181917428970337),
 (1525, 0.7131257057189941),
 (4492, 0.7130693197250366),
 (1338, 0.7090390920639038),
 (3211, 0.708549439907074),
 (1611, 0.7081342935562134)]

In [ ]:
# Function: convert similarity to relevance

def convert_to_relevance(num):
    num = num*100
    if num >= 90:
      return int(4)
    elif num>=80 and num < 90:
      return int(3)
    elif num>=70 and num <80:
      return int(2)
    elif num>=60 and num < 70:
      return int(1)
    else:
      return int(0)

In [ ]:
# List relevance
relevance = list(map(convert_to_relevance, [sim for docid, sim in sims]))
relevance[:10]

[3, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [ ]:
# List rank
rank = [docid for docid, sim in sims]
rank[:10]

[4058, 1626, 3635, 3540, 4891, 1525, 4492, 1338, 3211, 1611]

In [ ]:
# List job ad id based on rank
idx=rank
ad_id_rank = list(map(str, df.iloc[idx]["ad_id"]))
ad_id_rank[:10]

['38930962',
 '38855294',
 '38981356',
 '38854644',
 '38896974',
 '38910436',
 '38973549',
 '38996869',
 '38917147',
 '38967855']

In [ ]:
# Assemble the query results
top_num = 50
query_result = {"query_keywords": query, 
                "ad_ids": ad_id_rank[:top_num],
                "relevance": relevance[:top_num]
               }
print(query_result)

{'query_keywords': 'IT support', 'ad_ids': ['38930962', '38855294', '38981356', '38854644', '38896974', '38910436', '38973549', '38996869', '38917147', '38967855', '38870117', '38818674', '38964511', '38895262', '38902201', '38993423', '38890245', '38921684', '38932770', '38970801', '38981864', '38908425', '38862501', '38892500', '38915481', '38976112', '38868190', '38903729', '38968664', '38845985', '38945839', '38895558', '38855153', '38941800', '38868935', '38852205', '38940685', '38559932', '38932777', '38930456', '38936827', '38863632', '38951098', '38838768', '38999739', '38849819', '38823545', '38966935', '38913563', '38918574'], 'relevance': [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# Save query results

import json

#define the path for your json file
jf_path = '/content/drive/MyDrive/jobad/qry_rel_{}.json'.format(query)

# open your json file and add the dictionary
with open (jf_path, 'w') as j:
    json.dump(query_result, j)

# Model Evaluation

In [ ]:
# Get groud truth data ( Here we use example data as ground truth)

truth_df = pd.read_json('/content/drive/MyDrive/jobad/qry_rel_v1a.json', lines=True)
truth_df.head(5)

,query_keywords,ad_ids,relevance
0,it support,"[38464773, 38589022, 38935904, 38983386, 38898...","[4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
1,business analyst,"[38893545, 38989157, 38965055, 38967324, 38885...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
2,project manager,"[38840387, 38964511, 38905282, 38968664, 38863...","[4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
3,data analyst,"[38841804, 38897808, 38848695, 38966633, 38833...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, ..."
4,software engineer,"[38876123, 38789861, 38966902, 38994718, 38932...","[4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."


In [ ]:
# List job ad id most relevant to query keywords (ground truth)

y_true_ad_rank = truth_df[truth_df["query_keywords"]==query.lower()]["ad_ids"][0]
y_true_ad_rank[:5]

['38464773', '38589022', '38935904', '38983386', '38898601']

In [ ]:
# List relevance values corresponding to job ad with most relevance

y_true_relevance= truth_df[truth_df["query_keywords"]==query.lower()]["relevance"][0]
y_true_relevance[:5]

[4, 4, 4, 3, 3]

In [ ]:
# Prepare the information of the predicted job ad most relevant to query keywords

df_with_pred_score= df.iloc[rank][["ad_id", "title"]]
df_with_pred_score["rank"]=rank
df_with_pred_score["relevance"]=relevance

df_with_pred_score.head(5)

,ad_id,title,rank,relevance
4058,38930962,Recruitment Consultant - Information Technolog...,4058,3
1626,38855294,Java Developer / Full Stack Developer,1626,2
3635,38981356,Analyst/Senior Analyst Engineers(Security & De...,3635,2
3540,38854644,DevOps Engineer,3540,2
4891,38896974,Linux Systems Engineer,4891,2


In [ ]:
# Get the predicted relevance value based on ground-truth job ad rank

y_pred_relevance= [df_with_pred_score[df_with_pred_score["ad_id"]==int(x)]["relevance"].tolist()[0] for x in y_true_ad_rank]
y_pred_relevance[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# Calculate model evaluation based on relevance similarity

from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
import numpy as np

y_true=np.array(y_true_relevance)
y_pred=np.array(y_pred_relevance)
cosine = np.dot(y_true,y_pred)/(norm(y_true)*norm(y_pred))
print(f'The cosine similarity with truth data is: {cosine:.4f}')

The cosine similarity with truth data is: 0.2352


In [ ]:
# from sklearn.metrics import ndcg_score
# import numpy as np

# ndcg_score(y_true, y_pred)